In [1]:
import pandas as pd
import numpy as np
import os
import re
from PIL import Image
import h5py
from sklearn.model_selection import train_test_split
import argparse

In [2]:
import copy
import logging
import torch
import torch.nn as nn
from torchvision import *
import random
import sys
import time
import cv2

In [3]:
img_size = 512

output_path = 'train.h5'
img_dir =  r'ISIC2018_Task1-2_Training_Input/'
mask_dir = 'ISIC2018_Task2_Training_GroundTruth_v3/'

IMG_FILE_REGEX = r'ISIC_(\d+).jpg'
MASK_FILE_REGEX = r'ISIC_(\d+)_attribute_(.*).png'

ISIC2019_FILE_REGEX = r'ISIC_(\d+).jpg'

ATTR_TO_INDEX = {
    'globules' : 0,
    'milia_like_cyst' : 1,
    'negative_network' : 2,
    'pigment_network' : 3,
    'streaks' : 4,
}

In [4]:
Back = '../../../../../594A/Dataset/ISIC challenge 2018 Task 1-2/'

img_dir =  Back + img_dir
mask_dir = Back + mask_dir

#  ISIC2018 Val

In [5]:
df_ISIC2018 = pd.DataFrame(columns=['image', 'globules', 'milia_like_cyst', 
                                    'negative_network', 'pigment_network', 'streaks'])

In [6]:
df_ISIC2018.head()

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks


In [7]:
for img_file in os.listdir( img_dir ):
    if not img_file.endswith('.jpg'): continue
    img_id = str( re.search( IMG_FILE_REGEX, img_file ).group(1) )
    img_name = 'ISIC_' + img_id
    df_append = pd.DataFrame.from_records([{'image': img_name, 'globules': 2, 'milia_like_cyst': 2, 
                                      'negative_network': 2, 'pigment_network': 2, 'streaks': 2, 'pigment_network_pred': 2}])
    
    df_ISIC2018 = pd.concat([df_ISIC2018, df_append], ignore_index=True)

In [8]:
df_ISIC2018

,image,globules,milia_like_cyst,negative_network,pigment_network,streaks,pigment_network_pred
0,ISIC_0008913,2,2,2,2,2,2.0
1,ISIC_0000418,2,2,2,2,2,2.0
2,ISIC_0015614,2,2,2,2,2,2.0
3,ISIC_0000079,2,2,2,2,2,2.0
4,ISIC_0013399,2,2,2,2,2,2.0
...,...,...,...,...,...,...,...
2589,ISIC_0010334,2,2,2,2,2,2.0
2590,ISIC_0010493,2,2,2,2,2,2.0
2591,ISIC_0012813,2,2,2,2,2,2.0
2592,ISIC_0011115,2,2,2,2,2,2.0


## 2. save binary input 

In [9]:
'''
for mask_file in os.listdir( mask_dir ):
    if not mask_file.endswith('.png'): continue
    mask = Image.open( os.path.join( mask_dir, mask_file ) )
    mask = mask.resize( (img_size, img_size) )
    mask = np.array( mask )
    
    assert mask.shape == (img_size, img_size)
    assert mask.max() <= 255
    
    # decide feature exist or not
    if mask.max() > 0:
        binary_label = 1
    else:
        binary_label = 0
    
    m = re.search(MASK_FILE_REGEX, mask_file)
    
    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]
    
    if m.group(2) ==  'globules':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label
    elif m.group(2) ==  'milia_like_cyst':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label
    elif m.group(2) ==  'negative_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = binary_label
    elif m.group(2) ==  'pigment_network':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('pigment_network')]  = binary_label
    elif m.group(2) ==  'streaks':
        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('streaks')]  = binary_label
'''

"\nfor mask_file in os.listdir( mask_dir ):\n    if not mask_file.endswith('.png'): continue\n    mask = Image.open( os.path.join( mask_dir, mask_file ) )\n    mask = mask.resize( (img_size, img_size) )\n    mask = np.array( mask )\n    \n    assert mask.shape == (img_size, img_size)\n    assert mask.max() <= 255\n    \n    # decide feature exist or not\n    if mask.max() > 0:\n        binary_label = 1\n    else:\n        binary_label = 0\n    \n    m = re.search(MASK_FILE_REGEX, mask_file)\n    \n    file_idx = df_ISIC2018[df_ISIC2018['image'].str.contains(m.group(1))].index[0]\n    \n    if m.group(2) ==  'globules':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('globules')] = binary_label\n    elif m.group(2) ==  'milia_like_cyst':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('milia_like_cyst')]  = binary_label\n    elif m.group(2) ==  'negative_network':\n        df_ISIC2018.iloc[file_idx, df_ISIC2018.columns.get_loc('negative_network')]  = bina

In [10]:
#df_ISIC2018.to_csv('df_ISIC2018_val.csv', index=False)
df_ISIC2018 = pd.read_csv('dataframe_ISIC2018_train.csv')

In [11]:
#df = df_ISIC2018.loc[(df_ISIC2018.pigment_network == 0) & (df_ISIC2018.globules == 0)
#                    & (df_ISIC2018.milia_like_cyst == 0) & (df_ISIC2018.negative_network == 1) & (df_ISIC2018.streaks == 0)]

# Train.h5py

In [12]:
with h5py.File(output_path, "w") as f:
    count = len(df_ISIC2018)

    # store imgs to h5 file
    images = f.create_dataset( 'images', (count, 3, img_size, img_size), dtype=np.uint8 )
    image_ids = f.create_dataset( 'image_ids', (count,), dtype=int )
    img_id_to_h5idx = {}
    i = 0
    for idx in range(len(df_ISIC2018)):
        print( f'Processing image {i+1}/{count}' )
        img_file = df_ISIC2018.iloc[idx].image + '.jpg'
        img = Image.open( os.path.join( img_dir, img_file ) )
        img = img.resize( (img_size, img_size) )
        img = np.array( img )
        img = img.transpose(2, 0, 1)
        #(3,512,1024)
        assert img.shape == (3, img_size, img_size)
        assert img.max() <= 255
        images[i] = img
        img_id = int( re.search( IMG_FILE_REGEX, img_file ).group(1) )
        image_ids[i] = img_id
        img_id_to_h5idx[ img_id ] = i
        i += 1
        
    # store masks to h5 file
    i = 0
    masks = f.create_dataset('masks', (count, 5, img_size, img_size), dtype=np.uint8)
    attr_labels = f.create_dataset('labels', (count, 5), dtype=np.uint8)
    for mask_file in os.listdir( mask_dir ):
        if not mask_file.endswith('.png'): continue
        print( f'Processing mask {i+1}/{5*count}' )
        mask = Image.open( os.path.join( mask_dir, mask_file ) )
        mask = mask.resize( (img_size, img_size) )
        mask = np.array( mask )
        assert mask.shape == (img_size, img_size)
        assert mask.max() <= 255
        m = re.search(MASK_FILE_REGEX, mask_file)
        img_id, attr_id = int(m.group(1)), ATTR_TO_INDEX[ m.group(2) ]
        
        
        h5idx = img_id_to_h5idx[ img_id ]
        masks[ h5idx, attr_id ] = mask 
        if mask.max() > 0:
            attr_labels[h5idx, attr_id] = 1
        i += 1      

Processing image 1/2594
Processing image 2/2594
Processing image 3/2594
Processing image 4/2594
Processing image 5/2594
Processing image 6/2594
Processing image 7/2594
Processing image 8/2594
Processing image 9/2594
Processing image 10/2594
Processing image 11/2594
Processing image 12/2594
Processing image 13/2594
Processing image 14/2594
Processing image 15/2594
Processing image 16/2594
Processing image 17/2594
Processing image 18/2594
Processing image 19/2594
Processing image 20/2594
Processing image 21/2594
Processing image 22/2594
Processing image 23/2594
Processing image 24/2594
Processing image 25/2594
Processing image 26/2594
Processing image 27/2594
Processing image 28/2594
Processing image 29/2594
Processing image 30/2594
Processing image 31/2594
Processing image 32/2594
Processing image 33/2594
Processing image 34/2594
Processing image 35/2594
Processing image 36/2594
Processing image 37/2594
Processing image 38/2594
Processing image 39/2594
Processing image 40/2594
Processin

Processing image 323/2594
Processing image 324/2594
Processing image 325/2594
Processing image 326/2594
Processing image 327/2594
Processing image 328/2594
Processing image 329/2594
Processing image 330/2594
Processing image 331/2594
Processing image 332/2594
Processing image 333/2594
Processing image 334/2594
Processing image 335/2594
Processing image 336/2594
Processing image 337/2594
Processing image 338/2594
Processing image 339/2594
Processing image 340/2594
Processing image 341/2594
Processing image 342/2594
Processing image 343/2594
Processing image 344/2594
Processing image 345/2594
Processing image 346/2594
Processing image 347/2594
Processing image 348/2594
Processing image 349/2594
Processing image 350/2594
Processing image 351/2594
Processing image 352/2594
Processing image 353/2594
Processing image 354/2594
Processing image 355/2594
Processing image 356/2594
Processing image 357/2594
Processing image 358/2594
Processing image 359/2594
Processing image 360/2594
Processing i

Processing image 639/2594
Processing image 640/2594
Processing image 641/2594
Processing image 642/2594
Processing image 643/2594
Processing image 644/2594
Processing image 645/2594
Processing image 646/2594
Processing image 647/2594
Processing image 648/2594
Processing image 649/2594
Processing image 650/2594
Processing image 651/2594
Processing image 652/2594
Processing image 653/2594
Processing image 654/2594
Processing image 655/2594
Processing image 656/2594
Processing image 657/2594
Processing image 658/2594
Processing image 659/2594
Processing image 660/2594
Processing image 661/2594
Processing image 662/2594
Processing image 663/2594
Processing image 664/2594
Processing image 665/2594
Processing image 666/2594
Processing image 667/2594
Processing image 668/2594
Processing image 669/2594
Processing image 670/2594
Processing image 671/2594
Processing image 672/2594
Processing image 673/2594
Processing image 674/2594
Processing image 675/2594
Processing image 676/2594
Processing i

Processing image 955/2594
Processing image 956/2594
Processing image 957/2594
Processing image 958/2594
Processing image 959/2594
Processing image 960/2594
Processing image 961/2594
Processing image 962/2594
Processing image 963/2594
Processing image 964/2594
Processing image 965/2594
Processing image 966/2594
Processing image 967/2594
Processing image 968/2594
Processing image 969/2594
Processing image 970/2594
Processing image 971/2594
Processing image 972/2594
Processing image 973/2594
Processing image 974/2594
Processing image 975/2594
Processing image 976/2594
Processing image 977/2594
Processing image 978/2594
Processing image 979/2594
Processing image 980/2594
Processing image 981/2594
Processing image 982/2594
Processing image 983/2594
Processing image 984/2594
Processing image 985/2594
Processing image 986/2594
Processing image 987/2594
Processing image 988/2594
Processing image 989/2594
Processing image 990/2594
Processing image 991/2594
Processing image 992/2594
Processing i

Processing image 1262/2594
Processing image 1263/2594
Processing image 1264/2594
Processing image 1265/2594
Processing image 1266/2594
Processing image 1267/2594
Processing image 1268/2594
Processing image 1269/2594
Processing image 1270/2594
Processing image 1271/2594
Processing image 1272/2594
Processing image 1273/2594
Processing image 1274/2594
Processing image 1275/2594
Processing image 1276/2594
Processing image 1277/2594
Processing image 1278/2594
Processing image 1279/2594
Processing image 1280/2594
Processing image 1281/2594
Processing image 1282/2594
Processing image 1283/2594
Processing image 1284/2594
Processing image 1285/2594
Processing image 1286/2594
Processing image 1287/2594
Processing image 1288/2594
Processing image 1289/2594
Processing image 1290/2594
Processing image 1291/2594
Processing image 1292/2594
Processing image 1293/2594
Processing image 1294/2594
Processing image 1295/2594
Processing image 1296/2594
Processing image 1297/2594
Processing image 1298/2594
P

...

Processing mask 11782/12970
Processing mask 11783/12970
Processing mask 11784/12970
Processing mask 11785/12970
Processing mask 11786/12970
Processing mask 11787/12970
Processing mask 11788/12970
Processing mask 11789/12970
Processing mask 11790/12970
Processing mask 11791/12970
Processing mask 11792/12970
Processing mask 11793/12970
Processing mask 11794/12970
Processing mask 11795/12970
Processing mask 11796/12970
Processing mask 11797/12970
Processing mask 11798/12970
Processing mask 11799/12970
Processing mask 11800/12970
Processing mask 11801/12970
Processing mask 11802/12970
Processing mask 11803/12970
Processing mask 11804/12970
Processing mask 11805/12970
Processing mask 11806/12970
Processing mask 11807/12970
Processing mask 11808/12970
Processing mask 11809/12970
Processing mask 11810/12970
Processing mask 11811/12970
Processing mask 11812/12970
Processing mask 11813/12970
Processing mask 11814/12970
Processing mask 11815/12970
Processing mask 11816/12970
Processing mask 1181

Processing mask 12076/12970
Processing mask 12077/12970
Processing mask 12078/12970
Processing mask 12079/12970
Processing mask 12080/12970
Processing mask 12081/12970
Processing mask 12082/12970
Processing mask 12083/12970
Processing mask 12084/12970
Processing mask 12085/12970
Processing mask 12086/12970
Processing mask 12087/12970
Processing mask 12088/12970
Processing mask 12089/12970
Processing mask 12090/12970
Processing mask 12091/12970
Processing mask 12092/12970
Processing mask 12093/12970
Processing mask 12094/12970
Processing mask 12095/12970
Processing mask 12096/12970
Processing mask 12097/12970
Processing mask 12098/12970
Processing mask 12099/12970
Processing mask 12100/12970
Processing mask 12101/12970
Processing mask 12102/12970
Processing mask 12103/12970
Processing mask 12104/12970
Processing mask 12105/12970
Processing mask 12106/12970
Processing mask 12107/12970
Processing mask 12108/12970
Processing mask 12109/12970
Processing mask 12110/12970
Processing mask 1211

Processing mask 12371/12970
Processing mask 12372/12970
Processing mask 12373/12970
Processing mask 12374/12970
Processing mask 12375/12970
Processing mask 12376/12970
Processing mask 12377/12970
Processing mask 12378/12970
Processing mask 12379/12970
Processing mask 12380/12970
Processing mask 12381/12970
Processing mask 12382/12970
Processing mask 12383/12970
Processing mask 12384/12970
Processing mask 12385/12970
Processing mask 12386/12970
Processing mask 12387/12970
Processing mask 12388/12970
Processing mask 12389/12970
Processing mask 12390/12970
Processing mask 12391/12970
Processing mask 12392/12970
Processing mask 12393/12970
Processing mask 12394/12970
Processing mask 12395/12970
Processing mask 12396/12970
Processing mask 12397/12970
Processing mask 12398/12970
Processing mask 12399/12970
Processing mask 12400/12970
Processing mask 12401/12970
Processing mask 12402/12970
Processing mask 12403/12970
Processing mask 12404/12970
Processing mask 12405/12970
Processing mask 1240

Processing mask 12666/12970
Processing mask 12667/12970
Processing mask 12668/12970
Processing mask 12669/12970
Processing mask 12670/12970
Processing mask 12671/12970
Processing mask 12672/12970
Processing mask 12673/12970
Processing mask 12674/12970
Processing mask 12675/12970
Processing mask 12676/12970
Processing mask 12677/12970
Processing mask 12678/12970
Processing mask 12679/12970
Processing mask 12680/12970
Processing mask 12681/12970
Processing mask 12682/12970
Processing mask 12683/12970
Processing mask 12684/12970
Processing mask 12685/12970
Processing mask 12686/12970
Processing mask 12687/12970
Processing mask 12688/12970
Processing mask 12689/12970
Processing mask 12690/12970
Processing mask 12691/12970
Processing mask 12692/12970
Processing mask 12693/12970
Processing mask 12694/12970
Processing mask 12695/12970
Processing mask 12696/12970
Processing mask 12697/12970
Processing mask 12698/12970
Processing mask 12699/12970
Processing mask 12700/12970
Processing mask 1270

Processing mask 12963/12970
Processing mask 12964/12970
Processing mask 12965/12970
Processing mask 12966/12970
Processing mask 12967/12970
Processing mask 12968/12970
Processing mask 12969/12970
Processing mask 12970/12970
